In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape,test.shape

((159571, 8), (153164, 2))

In [4]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [5]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [6]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [7]:
test.isnull().sum()

id              0
comment_text    0
dtype: int64

In [8]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
classes = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [10]:
train[classes].sum()

obscene           8449
insult            7877
toxic            15294
severe_toxic      1595
identity_hate     1405
threat             478
dtype: int64

In [11]:
import matplotlib.pyplot as plt
train[classes].sum().plot(kind='bar',figsize=(10, 7));

In [12]:
x= train.iloc[:,2:].sum()
x

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [13]:
rowsums= train.iloc[:,2:].sum(axis=1)
rowsums

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Length: 159571, dtype: int64

In [14]:
rowsums= train.iloc[:,2:].sum(axis=1)
train['clean']= (rowsums==0)
train['clean'].sum()
print('Total Comments =', len(train))
print('Total Clean Comments =',train['clean'].sum())
print('Total Tags =', x.sum())

Total Comments = 159571
Total Clean Comments = 143346
Total Tags = 35098


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [16]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat((train, test)).comment_text 
all_text

C:\Users\ydeep\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
153159    . \n i totally agree, this stuff is nothing bu...
153160    == Throw from out field to home plate. == \n\n...
153161    " \n\n == Okinotorishima categories == \n\n I ...
153162    " \n\n == ""One of the founding nations of the...
153163    " \n :::Stop already. Your bullshit is not wel...
Name: comment_text, Length: 312735, dtype: object

In [17]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=30000)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)

union_vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

In [18]:
union_vectorizer.fit(all_text)
train_features = union_vectorizer.transform(train_text)
test_features = union_vectorizer.transform(test_text)

In [22]:
train_features.shape,test_features.shape

((159571, 60000), (153164, 60000))

In [19]:
class_scores = []
predictions_log_reg = pd.DataFrame.from_dict({'id': test['id'], 'text': test['comment_text']})
for class_name in classes:
    train_classes = train[class_name]
    logreg_classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        logreg_classifier, train_features, train_classes, cv=3, scoring='roc_auc'))
    class_scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    logreg_classifier.fit(train_features, train_classes)
    predictions_log_reg[class_name] = logreg_classifier.predict_proba(test_features)[:, 1]

print('Total Average CV score is {}'.format(np.mean(class_scores)))

CV score for class obscene is 0.9903319061397292
CV score for class insult is 0.9828863726047876
CV score for class toxic is 0.978800311010377
CV score for class severe_toxic is 0.9887814594255427
CV score for class identity_hate is 0.9831331098073733
CV score for class threat is 0.9887828384969798
Total Average CV score is 0.9854526662474651
